## Урок 4. Uplift-моделирование

**1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention**

**2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.**

In [28]:
!pip install scikit-uplift

In [29]:
!pip install causalml

Processing c:\users\максим\appdata\local\pip\cache\wheels\bc\d1\20\3807fc0990194f0acac1e142946c17cac35aa97267129dd64f\causalml-0.11.1-cp38-cp38-win_amd64.whl
  Using cached torch-1.9.0-cp38-cp38-win_amd64.whl (222.0 MB)
Processing c:\users\максим\appdata\local\pip\cache\wheels\fe\cd\78\a7e873cc049759194f8271f780640cf96b35e5a48bef0e2f36\pydotplus-2.0.2-py3-none-any.whl
  Using cached graphviz-0.17-py3-none-any.whl (18 kB)
  Using cached pygam-0.8.0-py2.py3-none-any.whl (1.8 MB)
  Using cached xgboost-1.4.2-py3-none-win_amd64.whl (97.8 MB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached pyro_ppl-1.7.0-py3-none-any.whl (678 kB)
  Using cached shap-0.37.0-cp38-cp38-win_amd64.whl (377 kB)
  Using cached progressbar2-3.53.3-py2.py3-none-any.whl (25 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached slicer-0.0.3-py3-none-any.whl (11 kB)
  Using cached python_utils-2.5.6-py2.py3-none-any.whl (

In [30]:
!pip install catboost

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds

In [58]:
import warnings
warnings.simplefilter('ignore')

In [39]:
df = pd.read_csv('data.csv', ',')

In [40]:
df.head(10)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
5,6,134.83,0,1,Surburban,0,Phone,Buy One Get One,1
6,9,280.20,1,0,Surburban,1,Phone,Buy One Get One,0
7,9,46.42,0,1,Urban,0,Phone,Buy One Get One,0
8,9,675.07,1,1,Rural,1,Phone,Discount,0
9,10,32.84,0,1,Urban,1,Web,Buy One Get One,0


In [41]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)
df.head(10)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0
5,6,134.83,0,1,Surburban,0,Phone,Buy One Get One,1
6,9,280.20,1,0,Surburban,1,Phone,Buy One Get One,0
7,9,46.42,0,1,Urban,0,Phone,Buy One Get One,0
8,9,675.07,1,1,Rural,1,Phone,Discount,0
9,10,32.84,0,1,Urban,1,Web,Buy One Get One,0


In [42]:
df['treatment'] = df['treatment'].apply(lambda x: 0 if x=='No Offer' else 1)
df.head(10)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0
5,6,134.83,0,1,Surburban,0,Phone,1,1
6,9,280.20,1,0,Surburban,1,Phone,1,0
7,9,46.42,0,1,Urban,0,Phone,1,0
8,9,675.07,1,1,Rural,1,Phone,1,0
9,10,32.84,0,1,Urban,1,Web,1,0


**3. сделать разбиение набора данных не тренировочную и тестовую выборки**

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
X_train, X_test = train_test_split(df.drop(['target'], axis=1), test_size=0.3, shuffle=True, random_state=42)
y_train, y_test = train_test_split(df['target'], test_size=0.3, shuffle=True, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((44800, 8), (44800,), (19200, 8), (19200,))

**4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)**

In [45]:
df.describe()

,recency,history,used_discount,used_bogo,is_referral,treatment,target
count,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000
mean,5.763734,242.085656,0.551031,0.549719,0.502250,0.667094,0.146781
std,3.507592,256.158608,0.497393,0.497526,0.499999,0.471257,0.353890
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,64.660000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,158.110000,1.000000,1.000000,1.000000,1.000000,0.000000
75%,9.000000,325.657500,1.000000,1.000000,1.000000,1.000000,0.000000
max,12.000000,3345.930000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
# Cмотрим значения
for value in ['recency','history','used_discount','used_bogo','zip_code','is_referral','channel','treatment','target']:
    print(df[value].value_counts(), '\n--------')

1     8952
10    7565
2     7537
9     6441
3     5904
4     5077
6     4605
5     4510
7     4078
11    3504
8     3495
12    2332
Name: recency, dtype: int64 
--------
29.99     7947
81.20        9
53.79        9
142.94       8
35.40        8
          ... 
701.66       1
246.45       1
798.83       1
125.19       1
104.00       1
Name: history, Length: 34833, dtype: int64 
--------
1    35266
0    28734
Name: used_discount, dtype: int64 
--------
1    35182
0    28818
Name: used_bogo, dtype: int64 
--------
Surburban    28776
Urban        25661
Rural         9563
Name: zip_code, dtype: int64 
--------
1    32144
0    31856
Name: is_referral, dtype: int64 
--------
Web             28217
Phone           28021
Multichannel     7762
Name: channel, dtype: int64 
--------
1    42694
0    21306
Name: treatment, dtype: int64 
--------
0    54606
1     9394
Name: target, dtype: int64 
--------


In [47]:
# Переводим все категориальные признаки в dummies
df = pd.concat([df.drop(['zip_code', 'channel'], axis=1), 
                          pd.get_dummies(df['zip_code'], prefix='zip_code'),
                          pd.get_dummies(df['channel'], prefix='channel')
                         ], 1)

df.head(10)

,recency,history,used_discount,used_bogo,is_referral,treatment,target,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
0,10,142.44,1,0,0,1,0,0,1,0,0,1,0
1,6,329.08,1,1,1,0,0,1,0,0,0,0,1
2,7,180.65,0,1,1,1,0,0,1,0,0,0,1
3,9,675.83,1,0,1,1,0,1,0,0,0,0,1
4,2,45.34,1,0,0,1,0,0,0,1,0,0,1
5,6,134.83,0,1,0,1,1,0,1,0,0,1,0
6,9,280.20,1,0,1,1,0,0,1,0,0,1,0
7,9,46.42,0,1,0,1,0,0,0,1,0,1,0
8,9,675.07,1,1,1,1,0,1,0,0,0,1,0
9,10,32.84,0,1,1,1,0,0,0,1,0,0,1


**5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями**

In [48]:
from sklift.models import SoloModel, ClassTransformation, TwoModels

In [49]:
from catboost import CatBoostClassifier

In [61]:
metrics_df = pd.DataFrame(columns=['uplift@10%', 'uplift@20%'])

In [62]:
cat_features = ['zip_code', 'channel']

In [63]:
sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_sm = sm.predict(X_test)
print(uplift_sm)

sm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)

[0.03216914 0.08452298 0.06661569 ... 0.06262959 0.03027332 0.06122132]


In [64]:
metrics_df = metrics_df.append({'uplift@10%': sm_score_10, 'uplift@20%': sm_score_20}, ignore_index=True)

In [65]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_ct = ct.predict(X_test)
print(uplift_ct)

ct_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)

[-0.84459557 -0.5492516  -0.64147218 ...  0.87426886 -0.81672069
 -0.75244202]


In [66]:
metrics_df = metrics_df.append({'uplift@10%': ct_score_10, 'uplift@20%': ct_score_20}, ignore_index=True)

In [67]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    method='vanilla'
)
tm = tm.fit(
    X_train, y_train, treat_train,
    estimator_trmnt_fit_params={'cat_features': cat_features}, 
    estimator_ctrl_fit_params={'cat_features': cat_features}
)

uplift_tm = tm.predict(X_test)
print(uplift_tm)

tm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.2)

[0.04196041 0.07958622 0.09562327 ... 0.05707612 0.04300713 0.07179594]


In [68]:
metrics_df = metrics_df.append({'uplift@10%': tm_score_10, 'uplift@20%': tm_score_20}, ignore_index=True)

**6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей**

In [69]:
metrics_df

,uplift@10%,uplift@20%
0,0.122913,0.100633
1,0.273450,0.227711
2,0.109515,0.091545
